In [1]:
import sys
import json

catchment_radius={'bus':'500','subway':'800','rail':'1000'}
footpaths = {'max_length':'1000', 'speed':'2.8','n_ntlegs':'2'}
params = {'catchment_radius':catchment_radius,'footpaths':footpaths}

default = {'training_folder': '../../scenarios/jakarta', 'params':params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)

{'training_folder': '../../scenarios/jakarta', 'params': {'catchment_radius': {'bus': '500', 'subway': '800', 'rail': '1000'}, 'footpaths': {'max_length': '1000', 'speed': '2.8', 'n_ntlegs': '2'}}}


In [2]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
import numpy as np
import random
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
from syspy.spatial.spatial import add_geometry_coordinates, nearest
from sklearn.neighbors import NearestNeighbors
from typing import Literal
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel

#num_cores = 1
print('numba threads',nb.config.NUMBA_NUM_THREADS)

on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
io_engine = 'pyogrio' if on_lambda else 'fiona'

numba threads 8


In [3]:
def get_epsg(lat: float, lon: float) -> int:
    '''
    return EPSG in meter for a given (lat,lon)
    lat is north south 
    lon is est west
    '''
    return int(32700 - round((45 + lat) / 90, 0) * 100 + round((183 + lon) / 6, 0))

# Folders stucture and params

everything is on S3 (nothing on ECR) so no direct input folder. just scenarios/{scen}/inputs/

In [4]:
base_folder = argv['training_folder']
input_folder = os.path.join(base_folder,'inputs/')
pt_folder  = os.path.join(input_folder,'pt/')
road_folder = os.path.join(input_folder,'road/')
od_folder =  os.path.join(input_folder,'od/')

output_folder = os.path.join(base_folder,'outputs/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
model_folder = os.path.join(input_folder, 'model/')


In [5]:
if 'footpaths' in argv['params'].keys():
    max_length = float(argv['params' ]['footpaths']['max_length'])
    speed = float(argv['params' ]['footpaths']['speed'])
    n_ntlegs = int(argv['params' ]['footpaths']['n_ntlegs'])
else:
    max_length = float(footpaths['max_length'])
    speed = float(footpaths['speed'])
    n_ntlegs = int(footpaths['n_ntlegs'])

# inputs

In [6]:
links = gpd.read_file(pt_folder + 'links.geojson',engine=io_engine) 
nodes = gpd.read_file(pt_folder + 'nodes.geojson',engine=io_engine)
links = links.set_index('index')
nodes = nodes.set_index('index')

In [7]:
rnodes_file = os.path.join(road_folder, 'road_nodes.geojson')
rnodes_file_provided = os.path.isfile(rnodes_file)
if rnodes_file_provided:
    rnodes = gpd.read_file(os.path.join(road_folder, 'road_nodes.geojson'), engine=io_engine)
    rnodes = rnodes.set_index('index')
    rlinks = gpd.read_file(os.path.join(road_folder, 'road_links.geojson'), engine=io_engine)
    rlinks = rlinks.set_index('index')
print('rnodes?',rnodes_file_provided)

rnodes? False


In [8]:
od_file = os.path.join(od_folder, 'od.geojson')
od_file_provided = os.path.isfile(od_file)
if od_file_provided:
    od_test = gpd.read_file(od_file, engine=io_engine)
    if 'name' not in od_test.columns:
        od_test['name'] = od_test['index']
    od_test['name'] = od_test['name'].fillna(od_test['index'].astype(str))
else:
    print('end of pathfinder')
    end_of_notebook

end_of_notebook here if OD_file not provided!!

In [9]:
od_test['geometry_o'] = od_test['geometry'].apply(lambda g: Point(g.coords[:][0]))
od_test['geometry_d'] = od_test['geometry'].apply(lambda g: Point(g.coords[:][1]))

od_test['origin'] = od_test['index'].astype(str) + '_o' 
od_test['destination'] = od_test['index'].astype(str) + '_d' 


zones = od_test.copy()
zones_d = od_test.copy()
zones['geometry'] = zones['geometry_o']
zones_d['geometry'] = zones_d['geometry_d']
zones['index'] = zones['origin']
zones_d['index'] = zones_d['destination']
zones = pd.concat([zones[['index','geometry']],zones_d[['index','geometry']]])
zones = zones.set_index('index')

od_set = set(zip(od_test['origin'], od_test['destination']))


# preparation

In [10]:
def add_col(links,col='length'):
    if col not in links.columns:
        return True
    elif any(links[col].isnull()):
        return True
    else:
        return False

In [11]:
if add_col(links,'length'):
    print('add length')
    centroid = [*LineString(nodes.centroid.values).centroid.coords][0]
    crs = get_epsg(centroid[1],centroid[0])
    print('create length from geometry')
    links['length'] = links.to_crs(crs).length
    
if add_col(links,'speed'):
    print('add speed')
    links['speed'] = links['length']/links['time'] * 3.6


add length
create length from geometry
add speed


# Walkmodel

In [12]:
sm = stepmodel.StepModel(epsg=4326)
sm.links = links
sm.nodes = nodes
sm.zones = zones

#sm.road_links = rlinks
#sm.road_nodes = rnodes

In [13]:
# We remove highway for the walkmodel
#walk_type = ['Artere','Collectrices','Locale','Nationale','Regionale']
#walkmodel.road_links = walkmodel.road_links[~walkmodel.road_links['highway'].isin(walk_type)]
#walkmodel.road_nodes = walkmodel.road_nodes.loc[list(np.unique(walkmodel.road_links[['a','b']].values.flatten()))]
#walkmodel.preparation_clusterize_nodes(distance_threshold=0.000002)
# Footpath between PT nodes
sm.preparation_footpaths(speed=speed, max_length=max_length, clusters_distance=0.00001)

In [14]:


# Access footpaths (zone_to_road and road_to_transit)
sm.preparation_ntlegs(
    short_leg_speed=speed,
    long_leg_speed=speed, # tout le monde marche
    threshold=1000,
    zone_to_transit=True,
    zone_to_road=False,
    road_to_transit=False,
    n_ntlegs=n_ntlegs,
    #max_ntleg_length=2000,
)

In [15]:
# Copy connectors to the model
sm.footpaths = sm.footpaths.drop(columns='voronoi')
#sm.footpaths = walkmodel.footpaths.drop(columns='voronoi').copy()
#sm.zone_to_transit = walkmodel.zone_to_transit.copy()

In [16]:
sm.footpaths['speed'] = sm.footpaths['length']/sm.footpaths['time'] * 3.6


# pathfinder

In [17]:
sm.step_pt_pathfinder(
    broken_routes=False,
    broken_modes=False,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    speedup=True,
    walk_on_road=False,
    path_analysis=False,
    engine='b', # b is faster
    od_set=od_set
)



start publicpathfinder
path_analysis


In [18]:
sm.analysis_pt_los(walk_on_road=False)

path_analysis: 100%|██████████████████████████| 12/12 [00:00<00:00, 8252.44it/s]


# create path

In [19]:
od_links = od_test.merge(sm.pt_los[['origin','destination','gtime','link_path']],on=['origin','destination'])
od_links = od_links.drop(columns = ['geometry','geometry_o','geometry_d','origin','destination'])
od_links = od_links.explode('link_path')
# may add route_width if quenedi can handle it one day.
od_links = od_links.merge(sm.links[['route_color','geometry','time','speed']],left_on='link_path',right_index=True)
od_links = od_links.drop(columns='link_path')

In [20]:
od_ntlegs = od_test.merge(sm.pt_los[['origin','destination','gtime','ntlegs']],on=['origin','destination'])
od_ntlegs = od_ntlegs.drop(columns = ['geometry','geometry_o','geometry_d','origin','destination'])
od_ntlegs = od_ntlegs.explode('ntlegs')

ntlegs_dict = sm.zone_to_transit.reset_index().set_index(['a','b'])['index'].to_dict()
od_ntlegs['ntlegs'] = od_ntlegs['ntlegs'].apply(ntlegs_dict.get)

od_ntlegs = od_ntlegs.merge(sm.zone_to_transit[['geometry','time','speed']],left_on='ntlegs',right_index=True)
od_ntlegs = od_ntlegs.drop(columns='ntlegs')
#od_ntlegs['route_color']='4B4B4B'

In [21]:
od_footpaths = od_test.merge(sm.pt_los[['origin','destination','gtime','footpaths']],on=['origin','destination'])
od_footpaths = od_footpaths.drop(columns = ['geometry','geometry_o','geometry_d','origin','destination'])
od_footpaths = od_footpaths.explode('footpaths')

footpaths_dict = sm.footpaths.reset_index().set_index(['a','b'])['index'].to_dict()
od_footpaths['footpaths'] = od_footpaths['footpaths'].apply(footpaths_dict.get)

od_footpaths = od_footpaths.merge(sm.footpaths[['geometry','time','speed']],left_on='footpaths',right_index=True)
od_footpaths = od_footpaths.drop(columns='footpaths')

In [22]:
od_route = pd.concat([od_links,od_footpaths,od_ntlegs],axis=0)

In [23]:
od_route['route_color'] = od_route['route_color'].fillna('838383')
od_route['route_color'] = '#' + od_route['route_color']

In [24]:
od_route = od_route.rename(columns={'name':'od_name'}).drop(columns='index')
od_route.reset_index(drop=True)
od_route.index.name='index'

In [25]:
od_route = gpd.GeoDataFrame(od_route,crs=4326)
od_route.to_file(output_folder+'od_route.geojson',driver='GeoJSON',engine=io_engine)

In [26]:
print('pathfinder successfull')

pathfinder successfull


# test

In [27]:
sm.links.to_file(output_folder+'test.geojson',driver='GeoJSON',engine=io_engine)

CPU times: user 483 ms, sys: 12.2 ms, total: 495 ms
Wall time: 493 ms
